In [1]:
import numpy as np
import pickle as pkl
import cv2, PIL, os
from cv2 import aruco
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from scipy.spatial.transform import Rotation as R
%matplotlib nbagg

In [2]:
mtx=np.load('datacalib_mtx_webcam.pkl', allow_pickle=True)
dist=np.load('datacalib_dist_webcam.pkl', allow_pickle=True)

In [5]:
size_of_marker =  0.0145
length_of_axis = 0.01
aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
parameters =  aruco.DetectorParameters_create()
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))
cap = cv2.VideoCapture(1)
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    frame_markers = aruco.drawDetectedMarkers(frame.copy(), corners, ids)
    rvecs,tvecs,trash = aruco.estimatePoseSingleMarkers(corners, size_of_marker , mtx, dist)
    imaxis = aruco.drawDetectedMarkers(frame.copy(), corners, ids)
    if tvecs is not None:
        for i in range(len(tvecs)):
            imaxis = aruco.drawAxis(imaxis, mtx, dist, rvecs[i], tvecs[i], length_of_axis)
            rvec=np.squeeze(rvecs[0], axis=None)
            tvec=np.squeeze(tvecs[0], axis=None)
            tvec=np.expand_dims(tvec, axis=1)
            rvec_matrix = cv2.Rodrigues(rvec)[0]
            proj_matrix = np.hstack((rvec_matrix,tvec))
            euler_angles = cv2.decomposeProjectionMatrix(proj_matrix)[6]
            cv2.putText(imaxis, 'X: '+str(int(euler_angles[0])),(20, 30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1.5,(0,0,255))
            cv2.putText(imaxis, 'Y: '+str(int(euler_angles[1])),(170, 30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1.5,(0,255,0))
            cv2.putText(imaxis, 'Z: '+str(int(euler_angles[2])),(320, 30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1.5,(255,0,0))
    cv2.imshow('Aruco',imaxis)
    #out.write(imaxis)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()